In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/EN/"

In [3]:
files = sorted(os.listdir(folder))
len(files)

1

In [4]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

48

In [5]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/48 [00:00<?, ?it/s]

In [6]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(48, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,Part. IV. New Philosophy. i 5 j From whence ma...,Part. IV. New Philosophy. i 5 j From whence ma...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ From whence ma...
1,"( 7* ) py to Postbumia , tnuill he had paid hi...","@( 7* ) py to Postbumia , tnuill he had paid h...","( 72 ) ny to Post@hmia , untill he had paid h..."
2,"3*8 Remarfy on Des-CartesV PartiV, bouse, shou...","@3*8 Remarfy on Des-Cartes@V Part@@iV@@@, bous...",318 Remarks on Des-Cartes's Part. IV. so hous...
3,Part. IV. New Philosophy* 267 Where are the Ey...,@@@@@Part. IV. New Philosophy* 267 Where are t...,267 Part. IV. New Philosophy@@@@. Where are t...
4,(»7) 5 Of ffeyres made upon condition of Marri...,@(@@»7@) 5 Of @@ffeyres made@ upon condition o...,"( 117 ) 6 Of an Heyre@ made, upon @an@@ absur..."


In [7]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,48.000000,48.000000,48.000000
mean,1309.854167,1373.770833,1372.770833
std,550.719297,586.360946,586.360946
min,24.000000,25.000000,24.000000
25%,1051.750000,1068.750000,1067.750000
50%,1272.500000,1291.000000,1290.000000
75%,1556.000000,1632.750000,1631.750000
max,2960.000000,3072.000000,3071.000000


In [8]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

127


In [9]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    48.000000
mean     21.225001
std      22.545614
min       1.796023
25%       4.732279
50%       9.316687
75%      28.526996
max      76.580959
Name: cer, dtype: float64

In [10]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    48.000000
mean     19.467190
std      20.032534
min       1.790281
25%       4.654360
50%       9.055790
75%      27.047168
max      70.427807
Name: cer, dtype: float64

In [11]:
data.to_pickle("../../data/en/data/test.pkl")